In [11]:
import pandas as pd
import requests
import re
import nltk
from html2text import HTML2Text
from bs4 import BeautifulSoup

import pymorphy2

## Серия преобразований

In [12]:
raw_table = pd.read_csv("result.csv", delimiter=";", names=["Ссылка", "Тема", "Тема2"])

In [13]:
raw_table.head(5)

,Ссылка,Тема,Тема2
0,medlawcenter.com.ua,Правові_послугиalltravels.com.ua,Дарвин
1,chevrolet-auto.ru,Шевроле,NaN
2,startubuntu.ru,Ubuntu,NaN
3,green-apple.com.ua,Веб-дизайн_и_разработка,NaN
4,stroim-nn.ru,Строительство_и_эксплуатация,NaN


In [14]:
raw_table.Тема.apply(lambda x: re.sub("[^а-яА-Я_]", " ", x).strip())

0                       Правов _послуги
1                               Шевроле
2                                      
3               Веб дизайн_и_разработка
4          Строительство_и_эксплуатация
5                              Здоровье
6                   Бытовая_электроника
7                              Животные
8                    Легковой_транспорт
9                                Физика
10                      Для_дома_и_сада
11                        Краски_и_лаки
12                          Государство
13             Посольства_и_консульства
14               Иконописные_мастерские
15                                Курсы
16                   Бизнес_и_экономика
17                               Теннис
18                        Чаты_и_форумы
19                Компьютеры_и_интернет
20       Выпечка_и_кондитерские_изделия
21                   Легковой_транспорт
22                                  КПК
23                     Продукты_питания
24         Строительство_и_эксплуатация


In [15]:
raw_table.Тема.value_counts()

Бизнес_и_экономика                                 3095
Новости_и_СМИ                                      1743
Торговля                                           1357
Путешествия_и_туризм                               1044
Образование                                        1018
Строительство_и_эксплуатация                        981
Здоровье                                            863
Государство                                         825
Недвижимость                                        731
Продукты_питания                                    695
Отдых_и_спорт                                       693
Компьютеры_и_интернет                               674
Общество_и_культура                                 672
Автомобили                                          656
Искусство_и_развлечения                             640
Мебель                                              577
Стройматериалы_и_инженерные_системы                 559
Организации                                     

In [16]:
t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")

# массив последних тем в иерархии (топики) и словарь, с пом. которого можно восстановить полную структуру
topics = [x[1:].split("/")[-1] for x in t]

In [17]:
k = 0
j = 0
a = []
for topic in topics:
    for_find = "_".join(topic.split()).capitalize()
    x = list(raw_table[raw_table.Тема == for_find].Ссылка)
    if len(x) > 0:
        a.append(x)
        k += 1
        print("YES", topic, len(x), "-", for_find)
    else:
        print("NO", topic)
        j += 1    

NO Наука и образование
YES наука 3 - Наука
NO математика
YES физика 4 - Физика
YES химия 108 - Химия
YES информатика 2 - Информатика
NO биоинформатика
NO анализ данных
YES литература 29 - Литература
YES образование 1018 - Образование
NO школьное
YES высшее 249 - Высшее
YES дополнительное 131 - Дополнительное
NO GoTo
YES Политика 46 - Политика
NO Внутренняя
NO Внешняя
NO Экономика и бизнес
YES Бизнес 30 - Бизнес
NO Стартапы
NO E-Contenta
YES Крупные компании 4 - Крупные_компании
NO Экономика
NO Отдых и развлечения
YES Кино 121 - Кино
YES Театр 3 - Театр
NO Компьютерные игры
YES Фитнес 147 - Фитнес
YES Медицина 4 - Медицина
NO Косметология


In [18]:
k, j

(15, 15)

### Нам подойдут: Бизнес и экономика, Новости и СМИ, Торговля, Путешествия и туризм, Образование,  Здоровье, Компьютеры и интернет, Химия, Кино, Фитнес, Высшее образование, Литература

In [69]:
def get_nouns_list_from_url(web_url):
    # получим текст
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, "lxml")
    
    h = HTML2Text()
    h.ignore_links = True
    
    raw_text = h.handle(soup.body.text)
    only_russian_text = re.sub("[^а-яА-Я_]", " ", raw_text).strip()
    
    morph = pymorphy2.MorphAnalyzer()
    
    from nltk.corpus import stopwords
    stoplist = stopwords.words('russian')
    
    # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форм
    words = only_russian_text.split()
    data = []
    for word in words:
        if 2 < len(word) < 16 and word not in stoplist:
            p = morph.parse(word)[0]
            if p.tag.POS == 'NOUN' or "VERB":
                data.append(str(p.normal_form))
    
    return data

In [10]:
# чтоб не попадались всякие xn--b1adgibfvfjyf6j.xn--p1ai
p = re.compile("^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$")
def valid(web_url):
    return p.match(web_url)

### Начинаем выкачивать сайты по 100 штук каждой выбранной темы
#### Один сайт занимает около секунды, так что все сайты выкачать очень сложно (учитывая, что мы смотрим только главные страницы)

In [11]:
vld = ["http://" + a for a in raw_table[raw_table.Тема == "Торговля"].Ссылка if valid("http://" + a)]

In [13]:
from tqdm import tqdm_notebook
import time
res = {}

for theme in ["Бизнес_и_экономика", "Новости_и_СМИ", "Торговля", "Путешествия_и_туризм", "Образование",  "Здоровье", 
              "Компьютеры_и_интернет", "Химия", "Кино", "Фитнес", "Высшее", "Литература"]:
    res[theme] = []
    # Оставляем только валидные урлы
    vld = ["http://" + a for a in raw_table[raw_table.Тема == theme].Ссылка if valid("http://" + a)]
    
    # Не хочу запариваться с ловлей ошибок - не в продакшен же пускаем хд
    for link in tqdm_notebook(vld[:100]):
        try:
            nouns = get_nouns_list_from_url(link)
            res[theme].append(nouns)
            time.sleep(0.1)
        except requests.exceptions.ConnectionError as r:
            r.status_code = "Connection refused"
        except Exception as msg:
            print(msg)

'NoneType' object has no attribute 'text'



'NoneType' object has no attribute 'text'

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'

'NoneType' object has no attribute 'text'

Exceeded 30 redirects.
'NoneType' object has no attribute 'text'

'NoneType' object has no attribute 'text'

'NoneType' object has no attribute 'text'
('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect data check',))





In [14]:
pd.to_pickle(res, "res.p")

In [15]:
res["Путешествия_и_туризм"]

[['главный',
  'номер',
  'удобство',
  'сауна',
  'отдых',
  'рыбалка',
  'контакт',
  'сайт',
  'весь',
  'право',
  'добро',
  'пожаловать',
  'гостев',
  'дом',
  'гостев',
  'дом',
  'это',
  'гостиница',
  'живописный',
  'берег',
  'волга',
  'исторический',
  'центр',
  'город',
  'рыбинск',
  'предлагать',
  'вы',
  'отдых',
  'один',
  'два',
  'трехместный',
  'номер',
  'вид',
  'волга',
  'спасо',
  'преображенский',
  'собор',
  'хлебный',
  'биржа',
  'для',
  'вы',
  'удобство',
  'проживание',
  'доступный',
  'цена',
  'многий',
  'другой',
  'рад',
  'видеть',
  'вы',
  'наш',
  'гостиница',
  'это',
  'комфортабельный',
  'местный',
  'номер',
  'удобный',
  'круглосуточный',
  'бесплатный',
  'стоянка',
  'зона',
  'отдых',
  'сад',
  'беседка',
  'шашлык',
  'пруд',
  'номер',
  'один',
  'два',
  'трехместный',
  'номер',
  'повышенный',
  'комфортность',
  'номер',
  'оснастить',
  'необходимый',
  'гостиничный',
  'мебель',
  'кровать',
  'шкаф',
  'тумба',
  '

# Дубль 2: Берём данные по-другому

In [5]:
import pandas as pd
import re
import requests
import pymystem3
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords


stopwords = stopwords.words("russian")
mystem = pymystem3.Mystem()

In [2]:
raw_table = pd.read_csv("result.csv", delimiter=";", names=["Ссылка", "Тема", "Тема2"])
raw_table.Тема.apply(lambda x: re.sub("[^а-яА-Я_]", " ", x).strip())

0                       Правов _послуги
1                               Шевроле
2                                      
3               Веб дизайн_и_разработка
4          Строительство_и_эксплуатация
5                              Здоровье
6                   Бытовая_электроника
7                              Животные
8                    Легковой_транспорт
9                                Физика
10                      Для_дома_и_сада
11                        Краски_и_лаки
12                          Государство
13             Посольства_и_консульства
14               Иконописные_мастерские
15                                Курсы
16                   Бизнес_и_экономика
17                               Теннис
18                        Чаты_и_форумы
19                Компьютеры_и_интернет
20       Выпечка_и_кондитерские_изделия
21                   Легковой_транспорт
22                                  КПК
23                     Продукты_питания
24         Строительство_и_эксплуатация


In [ ]:
from tqdm import tqdm_notebook
import time
res = {}
regexp = RegexpTokenizer('[а-яА-Я_]{2,25}')

p = re.compile("^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$")
def valid(web_url):
    return p.match(web_url)

for theme in ["Бизнес_и_экономика", "Новости_и_СМИ", "Торговля", "Путешествия_и_туризм", "Образование",  "Здоровье", 
              "Компьютеры_и_интернет", "Химия", "Кино", "Фитнес", "Высшее", "Литература"]:
    res[theme] = []
    # Оставляем только валидные урлы
    vld = ["http://" + a for a in raw_table[raw_table.Тема == theme].Ссылка if valid("http://" + a)]
    
    # Не хочу запариваться с ловлей ошибок - не в продакшен же пускаем хд
    for link in tqdm_notebook(vld[:250]):
        try:
            # и комментарии этих comprehensions тоже излишни :)))
            raw = " ".join(BeautifulSoup(requests.get(link).content, "lxml").text.split())
            res[theme].append([b for b in mystem.lemmatize(" ".join([a.lower() for a in regexp.tokenize(raw)])) if b != " "])
            time.sleep(0.1)
        except requests.exceptions.ConnectionError as r:
            r.status_code = "Connection refused"
        except Exception as msg:
            print(msg)

Traceback (most recent call last):
  File "/Library/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 404, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/Library/anaconda/lib/python3.5/site-packages/requests/packages/urllib3/util/response.py", line 59, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
requests.packages.urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Content-Security-Policy : default-src \'self\' \'unsafe-inline\' \'unsafe-eval\' data: lady7.net *.yandex.ru https://yandex.ru https://*.yandex.ru *.yandex.net https://*.yandex.net yandex.st *.yandex.st https://yandex.st https://*.yandex.st https://*.yandex.ua yastatic.net https://yastatic.net *.yandexadexchange.net *.google.com https://*.google.ru https://*.google.com https://*.google.com.ua https://*.google.kz https://*.google.az *.liveinternet.ru *.google-analytic


('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check',))


In [ ]:
pd.to_pickle(res, "res_2.p")